`Dataframe` to `DocumentArray`...

In [6]:
from docarray import Document, DocumentArray
import pandas

from k_fold import data_selection

train_df, test_df = data_selection('C:/Users/lachy/Pictures/utkface/utkface/')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


c:\Users\lachy\Videos\LACHY\SRI Repo\bias-project-ML\debiased_clip\k_fold.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= gender_mapper[df['gender'][i]]
c:\Users\lachy\Videos\LACHY\SRI Repo\bias-project-ML\debiased_clip\k_fold.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= race_mapper[df['race'][i]]


Initializing `CLIP`...

In [8]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [7]:
import finetuner

finetuner.login()

`Race ` and ` Gender` Fine-tuning...

In [8]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'] + ' gender and ' + doc.tags['race'] + ' race.' )
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

📄 Document: 836a050c14a235e882fe7e8f8aa78014
└── 💠 Chunks
    ├── 📄 Document: 8ea59a514177094fbb2e36bc5765411f
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122152403… │
    │   │ tags      │ {'age': 100, 'gender': 'male', 'filepath':                       │
    │   │           │ 'C:/Users/lachy/Pictures/utkface/utkface/100_0_0_20170112215240… │
    │   │           │ 'filename': '100_0_0_20170112215240346.jpg.chip.jpg', 'race':    │
    │   │           │ 'white'}                                                         │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: 73fb569bdcc3be80707f8d28cf487e2f
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [44]:
len(gender_pairs)

18966

Training `CLIP` model...

In [46]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32', # fine-tune CLIP
    train_data=gender_pairs,   
    learning_rate=1e-5,
    loss='CLIPLoss'
)

Pushing a DocumentArray to Hubble under the name finetuner-dastorage-default-elated-easley-train ...


Output()

In [47]:
for entry in run.stream_logs():
    print(entry)

Output()

[05:13:12] INFO     Starting finetuner training run ...                                                  __main__.py:350
DEBUG    Found Jina AI Cloud authentication token                                             __main__.py:362
DEBUG    Running in online mode                                                               __main__.py:363
INFO     Reading config ...                                                                   __main__.py:370
DEBUG    Reading config from stream                                                           __main__.py:382
INFO     Parsing config ...                                                                   __main__.py:385
INFO     Config loaded 📜                                                                     __main__.py:389
INFO     Run name: elated-easley                                                              __main__.py:391
INFO     Experiment name: default                                                             __main__.py:392


Saving the artifact

In [2]:
artifact = run.save_artifact('/artifact/')

NameError: name 'run' is not defined

Saving encoders...

In [9]:
clip_text_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-vision')

Saving images embeddings...

In [10]:
for d in test_da:
    d.uri = d.tags['filepath']
images = finetuner.encode(model=clip_image_encoder, data=test_da)

Output()

Create query texts to evaluate the model...

In [11]:
race_labels = ['black', 'white', 'asian', 'indian', 'others']
race_tks = []
for label in race_labels:
    race_tks.append('A photo of a person of ' + label + ' race.')

sex_labels = ['male', 'female']
sex_tks = []
for label in sex_labels:
    sex_tks.append('A photo of a person of ' + label + ' gender.')

sex_tks



['A photo of a person of male gender.',
 'A photo of a person of female gender.']

Saving texts embeddings...

In [12]:
text_docs = DocumentArray()
for item in sex_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: 2fd85d6629bd757ea29a5260a32a6975
╭──────────────────┬───────────────────────────────────────────────────────────╮
│ Attribute        │ Value                                                     │
├──────────────────┼───────────────────────────────────────────────────────────┤
│ text             │ A photo of a person of male gender.                       │
╰──────────────────┴───────────────────────────────────────────────────────────╯

📄 Document: 30d9105ea0a4d6a279490e22404ec11c
╭─────────────────┬────────────────────────────────────────────────────────────╮
│ Attribute       │ Value                                                      │
├─────────────────┼────────────────────────────────────────────────────────────┤
│ text            │ A photo of a person of female gender.                      │
╰─────────────────┴────────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 2                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   2                False             │
│   id          ('str',)       2                False             │
│   text        ('str',)       2                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model...

In [13]:
images.match(text_docs, metric='cosine', limit=1)

## Detecting bias in finetuned `CLIP` results

## Sex (Disparate impact)

### Selection Rate (Positive results / N)

In [14]:
positive_males, positive_females = 0, 0

total_males, total_females = 0, 0

for i in range(len(images)):
   if images[i].tags['gender'] == 'male':
        total_males += 1
        if images[i].matches[0].text.split(' ')[-2] == 'male':
            positive_males += 1
   else:
        total_females += 1
        if images[i].matches[0].text.split(' ')[-2] == 'female':
            positive_females += 1
                
males_sr, females_sr = positive_males/ total_males, positive_females/ total_females

males_sr, females_sr
    

(0.9826472962066183, 0.997348652231551)

#### Disparate impact

In [15]:

# disparate impact ratio = underprivileged group SR / privileged group SR
disp_impact = females_sr / males_sr
disp_impact

1.0149609692935455

In [16]:
if disp_impact < 0.8:
    print('Disparate impact present in female group / male group')


## Sex (Equalized odds)

In [50]:
tp_males, tp_females, fn_males, fn_females, fp_males, fp_females = 0, 0, 0, 0, 0, 0

for i in range(len(images)):
    if images[i].tags['gender'] == 'male':
        if images[i].matches[0].text.split(' ')[-2] == 'male':
            tp_males += 1
        else:
            fp_females += 1
            fn_males += 1 # False negative (wrong no male prediction, in this case, equal to female false positive)
    elif images[i].tags['gender'] == 'female':
        if images[i].matches[0].text.split(' ')[-2] == 'female':
            tp_females += 1
        else:
            fp_males += 1
            fn_females += 1
                
males_tpr, females_tpr = tp_males/ (tp_males + fn_males), tp_females/ (tp_females + fn_females)

males_fpr, females_fpr = fp_males/ (fp_males + fn_males), fp_females/ (fp_females + fn_females)


#### True Positive Rates

In [51]:
males_tpr, females_tpr

(0.9826472962066183, 0.997348652231551)

In [38]:
if abs(males_tpr - females_tpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_tpr - females_tpr))

Equalized odds


#### False Positive Rates

In [39]:
males_fpr, females_fpr

(0.12244897959183673, 0.8775510204081632)

In [40]:
if abs(males_fpr - females_fpr) < 0.05:
    print('Equalized odds')
else:
    print('Not equalized odds')
    print(abs(males_fpr - females_fpr))

Not equalized odds
0.7551020408163265


In [53]:
age = []
for i in range(len(images)):
    if  images[i].tags['gender'] != images[i].matches[0].text.split(' ')[-2]:
        age.append(images[i].tags['filepath'].split('_')[1])

age

['C:/Users/lachy/Pictures/utkface/utkface/10',
 'C:/Users/lachy/Pictures/utkface/utkface/11',
 'C:/Users/lachy/Pictures/utkface/utkface/15',
 'C:/Users/lachy/Pictures/utkface/utkface/15',
 'C:/Users/lachy/Pictures/utkface/utkface/16',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/1',
 'C:/Users/lachy/Pictures/utkface/utkface/21',
 'C:/Users/lachy/Pictures/utkface/utkface/21',
 'C:/Users/lachy/Pictures/utkface/utkface/21',
 'C:/Users/lachy/Pictures/utkface/utkface/21',
 'C:/Users/lachy/Pictures/utkface/utkface/21',
 'C:/Users/lachy/Pictures/utkface/utkface/23',
 'C:/Users/lachy/Pictures/utkface/utkface/24',
 'C:/Users/lachy/Pictures/utkface/utkface/24',
 'C:/Users/lachy/Pictures/utkface/utkface/25',
 'C:/Users/lachy/Pictures/utkface/utkface/26',
 'C:/Users/lachy/Pi